In [150]:
# import libraries
import datetime
from datetime import datetime
from pandas_datareader import data as pdr
import yfinance as yf
import pandas as pd
import time
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import random
import warnings
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import numpy as np

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

In [151]:
results_df = pd.read_csv('exports/stock_financial_quarterly.csv')
results_df['stock_code'] = results_df['stock_code'].astype(str)
results_df['Date'] = pd.to_datetime(results_df['Date'])
results_df['capitalExpenditures'] = results_df['capitalExpenditures'].abs()
results_df = results_df.round(3)

In [152]:
print(results_df.dtypes)

Date                   datetime64[ns]
revenue                       float64
capitalExpenditures           float64
grossDividend                 float64
netIncome                     float64
operatingCashFlow             float64
operatingIncome               float64
preferredDividends            float64
sharesOutstanding             float64
totalEquity                   float64
stock_code                     object
dtype: object


In [153]:
import pandas as pd

# Read historical price data from the CSV file
historical_prices_df = pd.read_csv('exports/historicalprices.csv', low_memory=False)
historical_prices_df['Date'] = pd.to_datetime(historical_prices_df['Date'])
historical_prices_df['MonthPeriod'] = historical_prices_df['Date'].dt.to_period('M')
dfs_to_concat = []

# Create an empty DataFrame to store the results

unique_stock_codes = results_df['stock_code'].unique()

In [154]:
unique_stock_codes

array(['5250', '7167', '7086', '5198', '03028', '7131', '0218', '0122',
       '1481', '5281', '9148', '7191', '7146', '0181', '6599', '5139',
       '5185', '7145', '0258', '7315', '7078', '0209', '5238', '2658',
       '7609', '5116', '5115', '2674', '0079', '2488', '1163', '03051',
       '5269', '5127', '5293', '5120', '03011', '1015', '7031', '6351',
       '7083', '0048', '4758', '0226', '6556', '5082', '5568', '5088',
       '7090', '5015', '6432', '0119', '7214', '7181', '7007', '0038',
       '0068', '7722', '7129', '0159', '0105', '4057', '7162', '7054',
       '03032', '6399', '0072', '8176', '7048', '5130', '7099', '5302',
       '03037', '8885', '5204', '7579', '6888', '5106', '7120', '2305',
       '5021', '7005', '03012', '0098', '5258', '7251', '1899', '6602',
       '0187', '5190', '3239', '3395', '5196', '4219', '5248', '9814',
       '7668', '0263', '6173', '5932', '0195', '6998', '5032', '0179',
       '5069', '0168', '9288', '7036', '8133', '5254', '6297', '5100',


In [155]:
def assess_and_update_class(liststock):
    # Iterate through the list elements
    for i in range(4, len(liststock)+1):
        current_row = liststock[i-1]
        previous_rows = liststock[i - 4:i]  # Get the previous 4 quarters, including the current one

        # Check if the most recent date's class is 'A'
        if current_row['rClass'] == 'A':
            # Calculate the mean value of the previous 4 quarters' ratios
            avg_ratios = {
                'rEPS': sum(row['rEPS'] for row in previous_rows) / 4,
                'rPE': sum(row['rPE'] for row in previous_rows) / 4,
                'rDY': sum(row['rDY'] for row in previous_rows) / 4
            }
            print(avg_ratios['rEPS'])
            # Check if the average ratios meet your criteria
            if avg_ratios['rEPS'] > 0.1 and avg_ratios['rPE'] < 10.0 and avg_ratios['rDY'] > 0.02:
                # Update the class to 'S'
                current_row['rClass'] = 'S'

    return liststock

In [156]:
# Function to calculate the growth rate

calculated_resulting_list = []
resulting_df = pd.DataFrame(columns=['stock_code', 'rEPS', 'rROE', 'rOM', 'rPR', 'rFCF', 'rDate'])
for stock_code in unique_stock_codes:
    calculated_stock_df = pd.DataFrame()
    calculated_stock_list = []
    calculated_data = {}
    filtered_df = results_df[results_df['stock_code'] == stock_code]
    filtered_df = filtered_df.sort_values(by='Date', ascending=True)
    filtered_df = filtered_df.reset_index(drop=True)
    # print(filtered_df)
    print(f"Stock Code: {stock_code}")
    a_class = False
    
    loop_start = 4 if len(filtered_df) == 4 else 5

    for i in range(loop_start, len(filtered_df)+1):
        # Calculate the average of the last four quarters' capital expenditures
        average_shares = sum(filtered_df['sharesOutstanding'].iloc[i - 4:i]) / 4
        average_equity = sum(filtered_df['totalEquity'].iloc[i - 4:i]) / 4
        
        sum_income = sum(filtered_df['netIncome'].iloc[i - 4:i])
        sum_revenue = sum(filtered_df['revenue'].iloc[i - 4:i])
        sum_operating_income = sum(filtered_df['operatingIncome'].iloc[i - 4:i])
        sum_gross_dividend = sum(filtered_df['grossDividend'].iloc[i - 4:i])
        sum_operating_cash_flow = sum(filtered_df['operatingCashFlow'].iloc[i - 4:i])
        sum_capital_expenditures = sum(filtered_df['capitalExpenditures'].iloc[i - 4:i])
        
        # Calculate rPE and rDY
        rEPS = sum_income / average_shares if average_shares != 0 else 0
        rDPS = sum_gross_dividend / average_shares if average_shares != 0 else 0
        stock_code = stock_code  # You may omit this line if stock_code doesn't change
        
        rPE = 0  # Initialize rPE
        rDY = 0  # Initialize rDY
        
        try:
            # print(f"Current i {i}")
            last_recorded_date = historical_prices_df[
                (historical_prices_df['stock_code'] == stock_code) &
                (historical_prices_df['MonthPeriod'] == filtered_df['Date'].iloc[i-1].to_period('M'))
            ].sort_values(by='Date').iloc[-1]['Date']
            # print(f"Last Date: {last_recorded_date} \n")
            
            closing_price = historical_prices_df[
                (historical_prices_df['stock_code'] == stock_code) &
                (historical_prices_df['Date'] == last_recorded_date)
            ]['Close'].values[0]
            # print(f"Closing Price: {closing_price} \n")
            # Rest of your code that depends on last_recorded_date and closing_price
        except IndexError:
            print("IndexError: Single positional indexer is out of bounds.")
            break  # This line will exit the loop when an IndexError is encountered

        # Calculate rPE and rDY
        rPE = (closing_price / rEPS) 
        rDY = (rDPS / closing_price)
        
        # Create a dictionary with the calculated values
        calculated_data = {
            'stock_code': stock_code,
            'rEPS': rEPS,
            'rDPS': rDPS,
            'rROE': sum_income / average_equity if average_equity != 0 else 0,
            'rOM': sum_operating_income / sum_revenue if sum_revenue != 0 else 0,
            'rPR': rDPS / rEPS if rEPS != 0 else 0,
            'rFCF': (sum_operating_cash_flow - sum_capital_expenditures),
            'rDate': filtered_df['Date'].iloc[i-1],
            'rPE': rPE,
            'rDY': rDY
        }
        # Calculate the 'rClass' based on the choices
        dividend_condition = (calculated_data['rDY'] >= 0.02) & (calculated_data['rPR'] >= 0.1) & (calculated_data['rPR'] <= 0.75)
        foundation_condition = (calculated_data['rOM'] >= 0.1) & (calculated_data['rFCF'] >= 0) & (calculated_data['rPE'] <= 10) & (calculated_data['rROE'] >= 0.20) & (calculated_data['rEPS'] >= 0.1)

        if dividend_condition & foundation_condition:
            calculated_data['rClass'] = 'A'
            a_class = True
        elif foundation_condition:
            calculated_data['rClass'] = 'B'
        elif dividend_condition:
            calculated_data['rClass'] = 'C'
        else:
            calculated_data['rClass'] = 'D'
            
        # Append the dictionary to the list
        calculated_stock_list.append(calculated_data)
        calculated_stock_list = assess_and_update_class(calculated_stock_list)
        calculated_resulting_list.append(calculated_data)
        
    # Convert the list of dictionaries to a DataFrame
    resulting_stock_df = pd.DataFrame(calculated_stock_list)

    # Display the DataFrame in a nice table format in a Jupyter Notebook
    print(resulting_stock_df)
    print("\n")

# Convert the list of dictionaries to a DataFrame
resulting_df = pd.DataFrame(calculated_resulting_list)
    


# Display the final resulting DataFrame
resulting_df


Stock Code: 5250
  stock_code      rEPS      rDPS      rROE       rOM       rPR    rFCF  \
0       5250  0.083395  0.000027  0.854336  0.067503  0.000319  831.43   
1       5250  0.059594  0.000000  0.552773  0.055748  0.000000  972.77   
2       5250  0.052031  0.000000  0.448081  0.049431  0.000000  872.40   
3       5250  0.051853  0.000000  0.441245  0.048191  0.000000  764.72   

       rDate        rPE       rDY rClass  
0 2022-09-30  18.706049  0.000017      D  
1 2022-12-31  32.385576  0.000000      D  
2 2023-03-31  39.207587  0.000000      D  
3 2023-06-30  38.570798  0.000000      D  


Stock Code: 7167
  stock_code      rEPS      rDPS      rROE       rOM       rPR    rFCF  \
0       7167  0.105020  0.044999  0.083531  0.087847  0.428483   -9.04   
1       7167  0.108532  0.040025  0.085124  0.086518  0.368784 -112.05   
2       7167  0.118253  0.035018  0.091113  0.091599  0.296123  -63.64   
3       7167  0.111458  0.020029  0.084511  0.091372  0.179697  -25.27   

       

  stock_code      rEPS      rDPS      rROE       rOM       rPR     rFCF  \
0       5139  1.431828  0.484979  0.158865  0.549922  0.338713  2052.84   
1       5139  1.354158  0.484979  0.145312  0.494241  0.358141   998.68   
2       5139  1.636011  0.495006  0.169785  0.526082  0.302569   -65.84   
3       5139  1.386471  0.495006  0.140887  0.461958  0.357026  -373.19   

       rDate       rPE       rDY rClass  
0 2022-08-31  9.847552  0.034396      C  
1 2022-11-30  9.496678  0.037712      C  
2 2023-02-28  7.334913  0.041251      C  
3 2023-05-31  8.222312  0.043422      C  


Stock Code: 5185
  stock_code      rEPS      rDPS      rROE       rOM       rPR     rFCF  \
0       5185  0.637809  0.123716  0.133754  0.012413  0.193970  8503.03   
1       5185  0.539723  0.310091  0.113115  0.011882  0.574538  7013.40   
2       5185  0.533452  0.304861  0.111524  0.011561  0.571488  2700.04   
3       5185  0.509692  0.299805  0.106207  0.010580  0.588208  2489.17   

       rDate       

  stock_code      rEPS      rDPS      rROE       rOM       rPR    rFCF  \
0       5269  0.006914  0.033793  0.006809  0.057286  4.887781   95.15   
1       5269  0.112793  0.053000  0.108819  0.919220  0.469887  109.42   
2       5269  0.110966  0.052000  0.104884  0.879946  0.468614  111.23   
3       5269  0.108741  0.051000  0.100788  0.848214  0.469003  111.95   

       rDate        rPE       rDY rClass  
0 2022-09-30  61.471322  0.079513      D  
1 2022-12-31   3.280342  0.143243      C  
2 2023-03-31   4.370727  0.107216      C  
3 2023-06-30   4.138259  0.113333      C  


Stock Code: 5127
  stock_code      rEPS      rDPS      rROE       rOM       rPR    rFCF  \
0       5127  0.035030  0.038589  0.026886  0.256105  1.101594  133.61   
1       5127  0.021458  0.034053  0.016524  0.195633  1.586992  125.56   
2       5127  0.019748  0.034053  0.015262  0.183681  1.724382  123.97   
3       5127  0.012456  0.017602  0.009667  0.131531  1.413165  120.84   

       rDate        rPE 

  stock_code      rEPS  rDPS      rROE       rOM  rPR    rFCF      rDate  \
0       5088  0.032077   0.0  0.019870  0.253703  0.0   -9.48 2022-09-30   
1       5088  0.030300   0.0  0.018966  0.182248  0.0 -134.42 2022-12-31   
2       5088  0.029214   0.0  0.018483  0.166712  0.0 -149.23 2023-03-31   
3       5088  0.032422   0.0  0.020404  0.179598  0.0 -174.66 2023-06-30   

         rPE  rDY rClass  
0  31.175385  0.0      D  
1  39.273876  0.0      D  
2  41.417973  0.0      D  
3  36.395008  0.0      D  


Stock Code: 7090
  stock_code      rEPS      rDPS      rROE       rOM       rPR   rFCF  \
0       7090  0.121419  0.080061  0.169234  0.099940  0.659374  58.24   
1       7090  0.141077  0.020022  0.191941  0.104291  0.141923  49.14   
2       7090  0.152856  0.020008  0.202324  0.107545  0.130891  58.70   
3       7090  0.579618  0.000000  0.652273  0.110160  0.000000  75.91   

       rDate        rPE       rDY rClass  
0 2022-09-30  17.353088  0.037997      C  
1 2022-12-31 

  stock_code      rEPS  rDPS      rROE       rOM  rPR   rFCF      rDate  \
0       7054 -0.008864   0.0 -0.032949 -0.249191 -0.0  16.71 2022-09-30   
1       7054 -0.005682   0.0 -0.021233 -0.058366 -0.0   6.40 2022-12-31   
2       7054 -0.007606   0.0 -0.028627 -0.225895 -0.0   2.44 2023-03-31   
3       7054 -0.008167   0.0 -0.030974 -0.278890 -0.0  -6.43 2023-06-30   

         rPE  rDY rClass  
0 -11.281709  0.0      D  
1 -21.119360  0.0      D  
2 -15.119064  0.0      D  
3 -13.468980  0.0      D  


Stock Code: 03032
  stock_code      rEPS  rDPS      rROE       rOM  rPR   rFCF      rDate  \
0      03032  0.033546   0.0  0.221226  0.326087  0.0  10.79 2022-08-31   
1      03032  0.037919   0.0  0.235334  0.376796  0.0  10.43 2023-02-28   

         rPE  rDY rClass  
0  11.029485  0.0      D  
1   9.757674  0.0      D  


Stock Code: 6399
  stock_code      rEPS      rDPS      rROE       rOM       rPR     rFCF  \
0       6399  0.082650  0.060002  0.378235  0.177686  0.725973  2303

...

  stock_code      rEPS      rDPS      rROE       rOM       rPR     rFCF  \
0       4588  0.529757  0.088035  0.117810  0.038157  0.166179  1016.60   
1       4588  0.415051  0.141934  0.090809  0.038288  0.341967  1855.34   
2       4588  0.443511  0.141934  0.095925  0.035570  0.320023  1290.64   
3       4588  0.611586  0.111933  0.129229  0.049112  0.183021   754.55   

       rDate       rPE       rDY rClass  
0 2022-09-30  5.644095  0.029443      C  
1 2022-12-31  8.360417  0.040903      C  
2 2023-03-31  8.680723  0.036866      C  
3 2023-06-30  6.131597  0.029849      C  


Stock Code: 03017
  stock_code      rEPS  rDPS      rROE       rOM  rPR   rFCF      rDate  \
0      03017  0.013973   0.0  0.314099  0.226342  0.0 -10.48 2021-12-31   
1      03017  0.013495   0.0  0.281960  0.189544  0.0 -10.55 2022-06-30   
2      03017  0.001025   0.0  0.021333  0.069622  0.0  -5.94 2022-12-31   
3      03017 -0.009106   0.0 -0.199044 -0.033771 -0.0   4.01 2023-06-30   

          rPE  rDY

  stock_code      rEPS  rDPS      rROE       rOM  rPR    rFCF      rDate  \
0       5243 -0.008395   0.0 -0.029915 -0.081728 -0.0 -300.56 2022-09-30   
1       5243 -0.012221   0.0 -0.043437 -0.092156 -0.0 -373.35 2022-12-31   
2       5243 -0.004866   0.0 -0.017169  0.018338 -0.0 -343.23 2023-03-31   
3       5243  0.002506   0.0  0.008746  0.084816  0.0 -171.19 2023-06-30   

         rPE  rDY rClass  
0 -13.698623  0.0      D  
1 -12.274303  0.0      D  
2 -43.153477  0.0      D  
3  87.782030  0.0      D  


Stock Code: 4995
  stock_code      rEPS  rDPS      rROE       rOM  rPR   rFCF      rDate  \
0       4995  0.002928   0.0  0.010697  0.021251  0.0 -11.00 2022-09-30   
1       4995 -0.001607   0.0 -0.005878  0.010994 -0.0  -8.92 2022-12-31   
2       4995 -0.002178   0.0 -0.007983  0.006470 -0.0  -3.13 2023-03-31   
3       4995 -0.002535   0.0 -0.009312  0.007448 -0.0   1.07 2023-06-30   

          rPE  rDY rClass  
0  175.916463  0.0      D  
1 -401.476667  0.0      D  
2 -23

  stock_code      rEPS      rDPS      rROE       rOM       rPR    rFCF  \
0       9679  0.141599  0.003500  0.051741  0.189373  0.024716  378.98   
1       9679  0.124792  0.005003  0.045164  0.105955  0.040088  388.34   
2       9679  0.097690  0.005003  0.035111  0.090540  0.051210  383.01   
3       9679  0.065599  0.005003  0.023488  0.106844  0.076261  219.86   

       rDate       rPE       rDY rClass  
0 2022-09-30  2.895497  0.008536      D  
1 2022-12-31  3.205338  0.012507      D  
2 2023-03-31  4.350502  0.011771      D  
3 2023-06-30  6.250063  0.012202      D  


Stock Code: 0197
  stock_code      rEPS  rDPS      rROE       rOM  rPR   rFCF      rDate  \
0       0197  0.034107   0.0  0.168330  0.163749  0.0   6.69 2022-09-30   
1       0197  0.035998   0.0  0.171117  0.167191  0.0  49.96 2022-12-31   
2       0197  0.028508   0.0  0.131828  0.147063  0.0  49.48 2023-03-31   
3       0197  0.024041   0.0  0.108777  0.144789  0.0  42.14 2023-06-30   

        rPE  rDY rClass 

  stock_code      rEPS  rDPS      rROE       rOM  rPR   rFCF      rDate  \
0       0140 -0.002703   0.0 -0.047623 -0.079953 -0.0 -48.77 2022-09-30   
1       0140 -0.005081   0.0 -0.091555 -0.194019 -0.0 -37.29 2022-12-31   
2       0140 -0.007845   0.0 -0.146564 -0.258212 -0.0 -25.78 2023-03-31   
3       0140 -0.010645   0.0 -0.209287 -0.430325 -0.0 -10.70 2023-06-30   

        rPE  rDY rClass  
0 -7.398762  0.0      D  
1 -5.904905  0.0      D  
2 -3.823939  0.0      D  
3 -2.818139  0.0      D  


Stock Code: 0017
  stock_code      rEPS  rDPS      rROE       rOM  rPR   rFCF      rDate  \
0       0017  0.003223   0.0  0.137061  0.118992  0.0  -3.36 2022-09-30   
1       0017  0.004622   0.0  0.194903  0.099697  0.0  -8.95 2022-12-31   
2       0017 -0.000090   0.0 -0.004073  0.074192 -0.0 -15.18 2023-03-31   
3       0017  0.001813   0.0  0.088634  0.106135  0.0 -10.49 2023-06-30   

          rPE  rDY rClass  
0    7.757205  0.0      D  
1    9.736889  0.0      D  
2 -446.815000  

,stock_code,rEPS,rDPS,rROE,rOM,rPR,rFCF,rDate,rPE,rDY,rClass
0,5250,8.339548e-02,0.000027,8.543363e-01,0.067503,0.000319,831.43,2022-09-30,1.870605e+01,0.000017,D
1,5250,5.959443e-02,0.000000,5.527730e-01,0.055748,0.000000,972.77,2022-12-31,3.238558e+01,0.000000,D
2,5250,5.203075e-02,0.000000,4.480813e-01,0.049431,0.000000,872.40,2023-03-31,3.920759e+01,0.000000,D
3,5250,5.185270e-02,0.000000,4.412450e-01,0.048191,0.000000,764.72,2023-06-30,3.857080e+01,0.000000,D
4,7167,1.050202e-01,0.044999,8.353106e-02,0.087847,0.428483,-9.04,2022-09-30,1.285467e+01,0.033333,C
5,7167,1.085317e-01,0.040025,8.512429e-02,0.086518,0.368784,-112.05,2022-12-31,1.243877e+01,0.029648,C
6,7167,1.182533e-01,0.035018,9.111349e-02,0.091599,0.296123,-63.64,2023-03-31,1.107791e+01,0.026731,C
7,7167,1.114579e-01,0.020029,8.451051e-02,0.091372,0.179697,-25.27,2023-06-30,1.139443e+01,0.015771,D
8,7086,-2.387268e-03,0.000000,-1.470417e-02,-0.159314,-0.000000,1.74,2022-09-30,-4.817222e+01,0.000000,D
9,7086,2.652520e-04,0.000000,1.633129e-03,0.015730,0.000000,2.41,2022-12-31,5.089500e+02,0.000000,D


In [157]:
resulting_df_rounded = resulting_df.round(3)

# Specify the file path where you want to save the CSV file
csv_file_path = 'exports/stock_ratio_quarterly.csv'

# Use the to_csv() method to export the DataFrame to a CSV file
resulting_df_rounded.to_csv(csv_file_path, index=False)  # Set index=False to exclude the DataFrame index from the CSV

print(f"DataFrame saved to {csv_file_path}")

DataFrame saved to exports/stock_ratio_quarterly.csv


In [164]:
resulting_df = pd.read_csv('exports/stock_ratio_quarterly.csv')
# Convert the 'rDate' column to datetime for date comparison
resulting_df['rDate'] = pd.to_datetime(resulting_df['rDate'])
# Find the index of the latest dates for each stock code
latest_dates_index = resulting_df.groupby('stock_code')['rDate'].idxmax()

# Filter the DataFrame to include only the latest dates
latest_dates_df = resulting_df.loc[latest_dates_index]

# Display the resulting DataFrame
print(latest_dates_df)

     stock_code   rEPS   rDPS    rROE     rOM      rPR      rFCF      rDate  \
3220       0001  0.040  0.000   0.090   0.255    0.000      7.62 2023-06-30   
1733       0002  0.441  0.100   0.249   0.274    0.227    117.01 2023-06-30   
3509       0005  0.000  0.000   0.004   0.015    0.000     -4.08 2023-05-31   
2622       0006 -0.059  0.000  -0.147  -0.060   -0.000      8.88 2023-06-30   
2735       0007 -0.026  0.000  -0.263  -3.790   -0.000    -78.19 2023-06-30   
3671       0008  0.030  0.000   0.073   0.097    0.000    -26.34 2023-06-30   
1509       0010  0.009  0.000   0.086   0.100    0.000     20.98 2023-06-30   
441        0011  0.029  0.000   0.090   0.440    0.000     12.06 2023-06-30   
3391       0012  0.057  0.025   0.065   0.058    0.442    -62.60 2023-06-30   
3711       0017  0.002  0.000   0.089   0.106    0.000    -10.49 2023-06-30   
1794       0018 -0.033  0.000  -0.309  -3.768   -0.000    -67.92 2023-06-30   
2298       0020 -0.027  0.000  -0.181  -0.697   -0.0

In [166]:
stock_code_records = latest_dates_df[latest_dates_df['rClass'] == "A"]
stock_code_records = stock_code_records.reset_index(drop=True)
stock_code_records

,stock_code,rEPS,rDPS,rROE,rOM,rPR,rFCF,rDate,rPE,rDY,rClass
0,5031,1.603,0.294,0.886,0.309,0.183,625.37,2023-06-30,3.244,0.057,A


In [69]:


# Apply the function to your DataFrame
stock_code_records = assess_and_update_class(stock_code_records)

print(stock_code_records)

rEPS    0.25325
rPE     8.56925
rDY     0.08175
dtype: float64
  stock_code   rEPS   rDPS   rROE    rOM    rPR    rFCF       rDate    rPE  \
0       5248  0.202  0.133  0.375  0.102  0.657  451.96  2022-10-31  9.806   
1       5248  0.242  0.155  0.427  0.106  0.642  457.02  2023-01-31  8.971   
2       5248  0.263  0.220  0.443  0.107  0.839  278.77  2023-04-30  8.791   
3       5248  0.306  0.190  0.496  0.112  0.623  373.20  2023-07-31  6.709   

     rDY rClass  
0  0.067      A  
1  0.072      A  
2  0.095      B  
3  0.093    AAA  


In [75]:
historical_prices_df = pd.read_csv('exports/historicalprices.csv', low_memory=False)
print (historical_prices_df.shape)
last_recorded_date = historical_prices_df[
(historical_prices_df['stock_code'] == "5162") 
].sort_values(by='Date')['Date']
print(last_recorded_date)

(408498, 7)
393105    2022-01-03
393106    2022-01-04
393107    2022-01-05
393108    2022-01-06
393109    2022-01-07
393110    2022-01-10
393111    2022-01-11
393112    2022-01-12
393113    2022-01-13
393114    2022-01-14
393115    2022-01-17
393116    2022-01-19
393117    2022-01-20
393118    2022-01-21
393119    2022-01-24
393120    2022-01-25
393121    2022-01-26
393122    2022-01-27
393123    2022-01-28
393124    2022-01-31
393125    2022-02-03
393126    2022-02-04
393127    2022-02-07
393128    2022-02-08
393129    2022-02-09
393130    2022-02-10
393131    2022-02-11
393132    2022-02-14
393133    2022-02-15
393134    2022-02-16
393135    2022-02-17
393136    2022-02-18
393137    2022-02-21
393138    2022-02-22
393139    2022-02-23
393140    2022-02-24
393141    2022-02-25
393142    2022-02-28
393143    2022-03-01
393144    2022-03-02
393145    2022-03-03
393146    2022-03-04
393147    2022-03-07
393148    2022-03-08
393149    2022-03-09
393150    2022-03-10
393151    2022-03-11
3

In [ ]:
stock_code_records = resulting_df[resulting_df['stock_code'] == "5248"]
stock_code_records